In [ ]:
#@markdown ###Step 1: Install nataili
#@markdown Click the play button, confirm output contains no bad looking errors.

#@markdown Warnings to the effect of `WARNING: The following packages were previously imported in this runtime:` can be ignored.

#@markdown This will take around 5 minutes, depending on initial conditions of colab.

%cd /content/
!git clone https://github.com/tazlin/nataili.git --branch sss --single-branch
%cd /content/nataili/
!pip install xformers
!pip install -e .

In [ ]:
#@markdown ###Step 2: Setup nataili

%cd /content/nataili
from PIL import Image
from nataili.model_manager.super import ModelManager
from nataili.stable_diffusion.compvis import CompVis


mm = ModelManager(compvis=True, esrgan=True, gfpgan=True, codeformer=True)

if mm is not None and mm.load("GFPGAN"):
  print()
  print("-> Everything went OK. You can proceed.")


In [ ]:
#@markdown ###Step 3: Load model
#@markdown You can enter the "name" field of any `.ckpt` model from https://github.com/Haidra-Org/AI-Horde-image-model-reference/blob/main/stable_diffusion.json.
#@markdown \

#@markdown You can rerun this cell to load a different model.

#@markdown \
#@markdown Note that `.safetensors` models will not work.

#@markdown \


model_to_load = "Zeipher Female Model" #@param {type:"string"}

_loaded_models = list(mm.compvis.loaded_models.keys())
for models_loaded in _loaded_models:
  mm.unload_model(models_loaded)

if mm.load(model_to_load):
  print()
  print(f"-> Model '{model_to_load}' loaded successfully.")
else:
  print("-> Something went wrong. Try rerunning this cell, or if that does not work,")
  print("-> go to the file bar, select 'Runtime' and 'Disconnect and Delete Runtime'")
  print("-> and restart from the beginnning.")

In [ ]:
#@markdown ###Generate Images
#@markdown On the left bar of this window, is a folder icon. All images will be output to the folder `samples`.
from nataili.gfpgan import gfpgan
from matplotlib import pyplot as plt

import random

compvis = CompVis(
    model=mm.loaded_models[model_to_load],
    model_name=model_to_load,
    output_dir='/content/',
    disable_voodoo=True,
)

seed = 0 #@param {type:"integer"}
#@markdown *Set seed to 0 to have it be random. The seed used will be output below and will be in the filename*
if seed and seed < 0:
  seed = None
if not seed:
  seed = None
prompt = "" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
combined_prompt = prompt if not negative_prompt else f"{prompt}###{negative_prompt}"
width = 512 #@param {type:"integer"}
height = 768 #@param {type:"integer"}

steps = 50 #@param {type:"integer"}
sampler_name = "k_lms" #@param ["k_lms", "DDIM", "k_dpm_2_a", "k_dpm_2", "k_euler_a", "k_euler", "k_heun", "k_lms", "k_dpm_fast", "k_dpm_adaptive", "k_dpmpp_2s_a", "k_dpmpp_2m", "k_dpmpp_sde"]
cfg_scale = 8 #@param {type:"number"}
clip_skip = 6 #@param {type:"integer"}
karras = True #@param {type:"boolean"}
if karras:
  sampler_name = sampler_name + "_karras"

use_gfpgan = True #@param {type:"boolean"}
ddim_eta = 1 #@param {type:"number"}

#@markdown \

batches = 2 #@param {type:"integer"}
#@markdown *Batches sets how many generations to do. Each generation made this way will have the seed incremented by 1.*

if width % 64 != 0 or height % 64 != 0:
  print("Both width and height must be divisible by 64")
elif seed and seed > (2**32 - 1):
  print(f"Seed needs to be less than {2**32 - 1}")
else:
  gfpgan_model = gfpgan(mm.loaded_models["GFPGAN"], output_dir='/content/samples/gfpgan')
  for i in range(batches):
    if not seed or seed < 0:
      seed = random.randint(0, 2**32 - 1)
    compvis.generate(
        seed=seed,
        prompt=combined_prompt,
        width=width,
        height=height,
        ddim_steps=steps,
        sampler_name=sampler_name,
        cfg_scale=cfg_scale,
        clip_skip=clip_skip,
        ddim_eta=ddim_eta,
      )
    plt.figure()
    print(f"seed {i+1}: {seed}")
    plt.axis("off")
    plt.imshow(compvis.output_images[i])
    plt.show()
    if use_gfpgan:
      gfpgan_model(input_image=compvis.output_images[-1], strength=1)
      print("with gfpgan:")
      plt.figure()
      plt.axis("off")
      plt.imshow(gfpgan_model.output_images[i])
      plt.show()

    if seed:
      seed += 1




